In [1]:
import pandas as pd
import re

In [ ]:
from ekonlpy.tag import Mecab
mecab = Mecab()

In [ ]:
%pip install ekonlpy

In [18]:

df = pd.read_csv("minutes_stage1.csv", encoding="utf-8-sig")

# BODY만 추출
df_body = df[df["text_type"] == "BODY"].copy()

print(df_body.shape)
df_body.head()


(265, 6)


,meeting_date,release_date,text_date,text_type,text,source_type
1,2011-11-24,2012-01-13,2012-01-13,BODY,먼저 증권대차제도에 대하여 논의가 이루어졌음 일부 위원은 향후 증권대여를 활용해야 ...,minutes
4,2011-12-08,2012-01-25,2012-01-25,BODY,국내외 경제동향 과 관련하여 일부 위원은 경제성장률이 올해 뿐만 아니라 내년까지 연...,minutes
9,2011-12-22,2012-02-07,2012-02-07,BODY,일부 위원은 당행은 그동안 금융시장 여건 개선추세를 보아가며 글로벌 금융위기 대응과...,minutes
12,2011-12-29,2012-02-14,2012-02-14,BODY,먼저 관련부서에서는 본 의안과 관련하여 그 동안 위원들이 위원협의회 등에 서 밝힌 ...,minutes
16,2012-01-13,2012-02-28,2012-02-28,BODY,국내외 경제동향 과 관련하여 일부 위원은 최근 건설투자 등 일부 경제지표 가 지난해...,minutes


In [19]:
df_body["text_raw"] = df_body["text"].astype(str).fillna("").str.strip()

In [35]:
df_sent = (
    df_body
    .assign(sentence=df_body["text_raw"].astype(str).str.split("음"))
    .explode("sentence")
    .reset_index(drop=True)
)

df_sent["sentence"] = df_sent["sentence"].str.strip()
df_sent = df_sent[df_sent["sentence"] != ""]


In [36]:
negation_words = {"않", "못", "아니"}

def tokenize_ekonlpy(text):
    tokens = mecab.pos(text)

    result = []
    for word, pos in tokens:
        # 명사 / 동사 / 형용사 / 부사
        if (
            pos.startswith("N") or   # 명사
            pos.startswith("V") or   # 동사
            pos == "VA" or            # 형용사
            pos == "MAG" or           # 부사
            word in negation_words    # 부정어
        ):
            result.append(word)

    return result


In [37]:
df_sent["tokens"] = df_sent["sentence"].apply(tokenize_ekonlpy)

In [38]:
df_sent[["meeting_date", "release_date", "sentence", "tokens"]].head()


,meeting_date,release_date,sentence,tokens
0,2011-11-24,2012-01-13,먼저 증권대차제도에 대하여 논의가 이루어졌,"[먼저, 증권, 대차제도, 대하, 논의, 이루, 졌]"
1,2011-11-24,2012-01-13,일부 위원은 향후 증권대여를 활용해야 할 금융시장 상황에 대해 객관적인 기준을 사전...,"[일부위원, 향후, 증권, 대여, 활용, 할, 금융시장, 상황, 대해, 객관, 인,..."
2,2011-11-24,2012-01-13,다른 일부 위원은 당행이 증권을 차입하는 경우 증권대여기관은 어떤 혜택 을 가지게 ...,"[일부위원, 당, 행이, 증권, 차입, 경우, 증권, 대여, 기관, 혜택, 가지, ..."
3,2011-11-24,2012-01-13,이에 대해 동 위원은 증권차입제도가 당행에게는 RP매각규모를 탄력적으 로 조절할 수...,"[이, 대해, 동위원, 증권, 차입, 제도, 당행, RP, 매각, 규모, 탄력, 으..."
4,2011-11-24,2012-01-13,또 다른 일부 위원은 증권대여 담보대상 증권의 범위를 금통위가 확대할 수 있는 권한...,"[또, 일부위원, 증권, 대여, 담보, 대상, 증권, 범위, 금통위, 확대, 수, ..."


In [ ]:
# df_sent에서 meeting_date별 sentence 개수 분포 확인
#df_sent.groupby("meeting_date").size().sort_index(ascending=False).head(20)


meeting_date
2025-12-11      9
2025-11-27    241
2025-10-23    210
2025-09-25     10
2025-09-11     15
2025-08-28    206
2025-07-24      7
2025-07-10    200
2025-06-25     24
2025-05-29    198
2025-04-17    162
2025-03-27     12
2025-03-13     17
2025-02-25    223
2025-01-16    167
2024-12-24     23
2024-11-28    207
2024-10-11    194
2024-09-26      7
2024-09-12     14
dtype: int64